В рамках этого практического задания вам будет дан код с кратким описанием. Вам необходимо описать какие из SOLID-принципов в нём нарушаются и отрефакторить его, приведя его в соответствие SOLID-принципам.

P.S. Решение всех задач не единственное. Необходимость любого рефакторинга на соответствие SOLID-принципам безусловно зависит от контекста и не является 100% необходимостью.

0. **Управление товарами на складе** (0б, с разбором)

Система для управления товарами на складе. Класс `WarehouseManager` выполняет обработку всех операций: добавление товара, удаление товара, генерация отчетов и синхронизация данных с удаленным сервером.

Исходный код:

In [ ]:
import requests

class WarehouseManager:
    def __init__(self):
        self.items = {}

    def add_item(self, item_id, quantity):
        if item_id in self.items:
            self.items[item_id] += quantity
        else:
            self.items[item_id] = quantity
        print(f"Added {quantity} of item {item_id} to warehouse.")

    def remove_item(self, item_id, quantity):
        if item_id not in self.items or self.items[item_id] < quantity:
            raise ValueError("Insufficient stock")
        self.items[item_id] -= quantity
        print(f"Removed {quantity} of item {item_id} from warehouse.")

    def generate_report(self):
        report = "\n".join(f"{item_id}: {quantity}" for item_id, quantity in self.items.items())
        print("Warehouse Report:\n", report)
        return report

    def sync_with_server(self):
        response = requests.post("https://example.com/sync", json=self.items)
        if response.status_code == 200:
            print("Sync successful")
        else:
            print("Sync failed")

manager = WarehouseManager()
manager.add_item("item1", 10)
manager.remove_item("item1", 5)
manager.generate_report()
manager.sync_with_server()


Added 10 of item item1 to warehouse.
Removed 5 of item item1 from warehouse.
Warehouse Report:
 item1: 5
Sync failed


Проблемы:

1. **Нарушение SRP**:
   - `WarehouseManager` занимается управлением запасами, генерацией отчетов и синхронизацией с сервером.
   - Каждая из этих задач требует изменений в одном классе.

2. **Нарушение OCP**:
   - Если нужно добавить новую функциональность, например, расчет стоимости товаров, потребуется изменить `WarehouseManager`.

3. **Нарушение DIP**:
   - Класс напрямую зависит от библиотеки `requests` для синхронизации.

Решение:

In [ ]:
class Inventory:
    def __init__(self):
        self.items = {}

    def add_item(self, item_id, quantity):
        if item_id in self.items:
            self.items[item_id] += quantity
        else:
            self.items[item_id] = quantity

    def remove_item(self, item_id, quantity):
        if item_id not in self.items or self.items[item_id] < quantity:
            raise ValueError("Insufficient stock")
        self.items[item_id] -= quantity

    def get_items(self):
        return self.items

class ReportGenerator:
    @staticmethod
    def generate_report(items):
        report = "\n".join(f"{item_id}: {quantity}" for item_id, quantity in items.items())
        print("Warehouse Report:\n", report)
        return report

class ServerSync:
    def __init__(self, server_url, client):
        self.server_url = server_url
        self.client = client

    def sync(self, data):
        response = self.client.post(self.server_url, json=data)
        if response.status_code == 200:
            print("Sync successful")
        else:
            print("Sync failed")

import requests

# Используем класс Inventory
inventory = Inventory()
inventory.add_item("item1", 10)
inventory.add_item("item2", 5)
inventory.remove_item("item1", 3)

# Генерация отчета
report = ReportGenerator.generate_report(inventory.get_items())

# Синхронизация с сервером
server_sync = ServerSync("https://example.com/sync", requests)
server_sync.sync(inventory.get_items())

Warehouse Report:
 item1: 7
item2: 5
Sync failed


1. **Система управления подписками (2б)**

В системе управления подписками есть разные планы подписок: базовый, премиум и бизнес. Каждому плану соответствует своя логика расчета стоимости с учетом скидок и налогов.

Исходный код:

In [ ]:
class SubscriptionManager:
    def __init__(self):
        self.plans = {}

    def add_plan(self, name, cost):
        self.plans[name] = {"cost": cost}

    def calculate_price(self, plan_name, tax_rate, discount=0):
        if plan_name not in self.plans:
            raise ValueError("Plan not found")
        base_price = self.plans[plan_name]["cost"]
        if plan_name == "premium":
            discount += 0.1  # Дополнительная скидка для премиум-плана
        elif plan_name == "business":
            discount += 0.2  # Дополнительная скидка для бизнес-плана
        final_price = base_price * (1 - discount) * (1 + tax_rate)
        return final_price

manager = SubscriptionManager()
manager.add_plan("basic", 10)
manager.add_plan("premium", 20)
manager.add_plan("business", 30)

print(manager.calculate_price("basic", 0.2))
print(manager.calculate_price("premium", 0.2))
print(manager.calculate_price("business", 0.2))


12.0
21.599999999999998
28.799999999999997


**Проблемы:**

1. **Нарушение SRP:**
   - Логика расчёта цены для разных типов подписок сосредоточена в одном методе.

2. **Нарушение OCP:**
   - Добавление нового типа подписки приведёт к изменению метода `calculate_price`.

3. **Нарушение DIP:**
   - `SubscriptionManager` зависит от конкретной реализации расчета цены.

**Решение:**

In [ ]:
from abc import ABC, abstractmethod

class SubscriptionPlan(ABC):
    """Base class for subscription plans"""
    def __init__(self, cost):
        self.cost = cost

    @abstractmethod
    def calculate_price(self, tax_rate, discount=0):
        pass

class BasicPlan(SubscriptionPlan):
    def calculate_price(self, tax_rate, discount=0):
        return self.cost * (1 - discount) * (1 + tax_rate)

class PremiumPlan(SubscriptionPlan):
    def calculate_price(self, tax_rate, discount=0):
        discount += 0.1
        return self.cost * (1 - discount) * (1 + tax_rate)

class BusinessPlan(SubscriptionPlan):
    def calculate_price(self, tax_rate, discount=0):
        discount += 0.2
        return self.cost * (1 - discount) * (1 + tax_rate)

class SubscriptionManager:
    def __init__(self):
        self.plans = {}

    def add_plan(self, name, plan):
        self.plans[name] = plan

    def calculate_price(self, plan_name, tax_rate, discount=0):
        if plan_name not in self.plans:
            raise ValueError("Plan not found")
        return self.plans[plan_name].calculate_price(tax_rate, discount)


manager = SubscriptionManager()
manager.add_plan("basic", BasicPlan(10))
manager.add_plan("premium", PremiumPlan(20))
manager.add_plan("business", BusinessPlan(30))

print(manager.calculate_price("basic", 0.2))
print(manager.calculate_price("premium", 0.2))
print(manager.calculate_price("business", 0.2))


12.0
21.599999999999998
28.799999999999997


2. **Система бронирования билетов (2б)**

В системе бронирования билетов есть разные типы билетов: эконом (невозвратные) и бизнес (возвратные). Каждый тип билета имеет свою стоимость и условия возврата.

Исходный код:

In [ ]:
class TicketBookingSystem:
    def __init__(self):
        self.tickets = []

    def book_ticket(self, ticket_type, cost, refundable):
        self.tickets.append({"type": ticket_type, "cost": cost, "refundable": refundable})
        print(f"Booked {ticket_type} ticket for ${cost}. Refundable: {refundable}")

    def process_refund(self, ticket_id):
        ticket = self.tickets[ticket_id]
        if not ticket["refundable"]:
            raise ValueError("This ticket is non-refundable")
        print(f"Refund processed for {ticket['type']} ticket costing ${ticket['cost']}")

    def charge_payment(self, amount):
        print(f"Charging ${amount} through payment gateway...")

system = TicketBookingSystem()
system.book_ticket("economy", 100, False)
system.book_ticket("business", 300, True)
system.process_refund(1)
system.charge_payment(400)


Booked economy ticket for $100. Refundable: False
Booked business ticket for $300. Refundable: True
Refund processed for business ticket costing $300
Charging $400 through payment gateway...


**Проблемы:**

1. **Нарушение SRP:**
   - Один класс выполняет бронирование, обработку возвратов и оплату.

2. **Нарушение OCP:**
   - Добавление новых типов билетов приведёт к изменениям в методах класса.

3. **Нарушение ISP:**
   - Методы `process_refund` и `charge_payment` не используются одновременно всеми билетами.

4. **Нарушение DIP:**
   - Система напрямую взаимодействует с оплатой, что делает её жестко зависимой от конкретной реализации.

**Решение:**

In [ ]:
from abc import ABC, abstractmethod

class Ticket(ABC):
    @abstractmethod
    def book(self):
        pass

    @abstractmethod
    def process_refund(self):
        pass

class EconomyTicket(Ticket):
    def __init__(self, cost):
        self.cost = cost

    def book(self):
        print(f"Booked economy ticket for ${self.cost}. Non-refundable.")

    def process_refund(self):
        raise ValueError("Economy tickets are non-refundable")

class BusinessTicket(Ticket):
    def __init__(self, cost):
        self.cost = cost

    def book(self):
        print(f"Booked business ticket for ${self.cost}. Refundable.")

    def process_refund(self):
        print(f"Refund processed for business ticket costing ${self.cost}")

class PaymentGateway:
    def charge_payment(self, amount):
        print(f"Charging ${amount} through payment gateway...")

class TicketBookingSystem:
    def __init__(self, payment_gateway):
        self.payment_gateway = payment_gateway
        self.tickets = []

    def book_ticket(self, ticket):
        ticket.book()
        self.tickets.append(ticket)

    def process_refund(self, ticket_id):
        self.tickets[ticket_id].process_refund()

    def charge_payment(self, amount):
        self.payment_gateway.charge_payment(amount)

# Создаем платежный шлюз
payment_gateway = PaymentGateway()

# Создаем систему бронирования
system = TicketBookingSystem(payment_gateway)

# Бронируем билеты
economy_ticket = EconomyTicket(100)
business_ticket = BusinessTicket(300)

system.book_ticket(economy_ticket)
system.book_ticket(business_ticket)

# Обрабатываем возврат
system.process_refund(1)

# Оплата
system.charge_payment(400)


Booked economy ticket for $100. Non-refundable.
Booked business ticket for $300. Refundable.
Refund processed for business ticket costing $300
Charging $400 through payment gateway...


3. **Система управления бронированием для отелей (2б)**

В системе управления бронированием для отелей обрабатываются все аспекты бронирования: проверка доступности номеров, расчет стоимости, создание бронирования, отправка уведомлений клиенту и генерация отчетов для менеджеров.

Исходный код:

In [ ]:
import requests

class BookingManager:
    def __init__(self):
        self.rooms = {"101": True, "102": False, "103": True}  # True - свободно, False - занято

    def check_availability(self, room_id):
        return self.rooms.get(room_id, False)

    def calculate_price(self, room_id, nights):
        base_price = 100
        if room_id == "102":
            base_price = 150  # Цена люкса
        return base_price * nights

    def create_booking(self, room_id, customer_name, nights):
        if not self.check_availability(room_id):
            raise ValueError("Room not available")
        price = self.calculate_price(room_id, nights)
        self.rooms[room_id] = False
        print(f"Booking created for {customer_name} in room {room_id} for {nights} nights. Total price: ${price}.")
        return {"room_id": room_id, "customer_name": customer_name, "nights": nights, "price": price}

    def send_notification(self, customer_name, email):
        print(f"Sending booking confirmation to {customer_name} at {email}...")
        response = requests.post("https://example.com/notify", json={"name": customer_name, "email": email})
        if response.status_code == 200:
            print("Notification sent.")
        else:
            print("Failed to send notification.")

    def generate_report(self):
        booked_rooms = [room for room, available in self.rooms.items() if not available]
        print("Booked rooms:", booked_rooms)
        return booked_rooms

manager = BookingManager()
manager.create_booking("101", "Alice", 3)
manager.send_notification("Alice", "alice@example.com")
manager.generate_report()


Booking created for Alice in room 101 for 3 nights. Total price: $300.
Sending booking confirmation to Alice at alice@example.com...
Failed to send notification.
Booked rooms: ['101', '102']


['101', '102']

**Проблемы:**

1. **Нарушение SRP:**
   - Один класс обрабатывает доступность номеров, расчёт цены, создание бронирования, отправку уведомлений и генерацию отчётов.

2. **Нарушение DIP:**
   - Прямая зависимость от `requests`.

**Решение:**

In [ ]:
class RoomManager:
    def __init__(self):
        self.rooms = {"101": True, "102": False, "103": True}  # True - available, False - booked

    def check_availability(self, room_id):
        return self.rooms.get(room_id, False)

    def book_room(self, room_id):
        if not self.check_availability(room_id):
            raise ValueError("Room not available")
        self.rooms[room_id] = False

    def generate_report(self):
        booked_rooms = [room for room, available in self.rooms.items() if not available]
        print("Booked rooms:", booked_rooms)
        return booked_rooms

class PriceCalculator:
    def calculate_price(self, room_id, nights):
        base_price = 100
        if room_id == "102":
            base_price = 150
        return base_price * nights

class NotificationService:
    def send_notification(self, customer_name, email):
        print(f"Sending booking confirmation to {customer_name} at {email}...")
        response = requests.post("https://example.com/notify", json={"name": customer_name, "email": email})
        if response.status_code == 200:
            print("Notification sent.")
        else:
            print("Failed to send notification.")

class BookingService:
    def __init__(self, room_manager, price_calculator):
        self.room_manager = room_manager
        self.price_calculator = price_calculator

    def create_booking(self, room_id, customer_name, nights):
        self.room_manager.book_room(room_id)
        price = self.price_calculator.calculate_price(room_id, nights)
        print(f"Booking created for {customer_name} in room {room_id} for {nights} nights. Total price: ${price}.")
        return {"room_id": room_id, "customer_name": customer_name, "nights": nights, "price": price}

# Usage
room_manager = RoomManager()
price_calculator = PriceCalculator()
booking_service = BookingService(room_manager, price_calculator)
notification_service = NotificationService()

booking = booking_service.create_booking("101", "Alice", 3)
notification_service.send_notification("Alice", "alice@example.com")
room_manager.generate_report()

Booking created for Alice in room 101 for 3 nights. Total price: $300.
Sending booking confirmation to Alice at alice@example.com...
Failed to send notification.
Booked rooms: ['101', '102']


['101', '102']

4. **Система управления логистикой (1б)**

В системе управления логистикой есть разные типы заказов: стандартные, экспресс и крупногабаритные. Каждый тип заказа имеет свои уникальные требования:  
- **Стандартные заказы** рассчитывают маршрут на основе ближайших пунктов доставки.  
- **Экспресс-заказы** имеют фиксированные маршруты, но для них также требуется учитывать время доставки.  
- **Крупногабаритные заказы** требуют предварительного расчета вместимости транспорта.

Исходный код:

In [ ]:
class Order:
    def __init__(self, order_id, destination, weight):
        self.order_id = order_id
        self.destination = destination
        self.weight = weight

    def calculate_route(self):
        # Универсальный метод расчета маршрута
        print(f"Calculating generic route for order {self.order_id} to {self.destination}")

    def calculate_delivery_time(self):
        return 2  # Время в днях

    def calculate_capacity(self):
        return self.weight

class StandardOrder(Order):
    def calculate_route(self):
        print(f"Calculating optimized route for standard order {self.order_id} to {self.destination}")


class ExpressOrder(Order):
    def calculate_delivery_time(self):
        print(f"Calculating delivery time for express order {self.order_id}")
        return 1  # Экспресс-доставка за 1 день


class BulkOrder(Order):
    def calculate_capacity(self):
        print(f"Calculating capacity for bulk order {self.order_id}")
        return self.weight * 2  # Специальный расчет для крупногабаритных заказов

class LogisticsManager:
    def __init__(self):
        self.orders = []

    def add_order(self, order):
        self.orders.append(order)

    def process_orders(self):
        for order in self.orders:
            order.calculate_route()
            delivery_time = order.calculate_delivery_time()
            capacity = order.calculate_capacity()
            print(f"Order {order.order_id}: delivery time is {delivery_time} days, capacity {capacity}")

manager = LogisticsManager()
manager.add_order(StandardOrder("001", "New York", 10))
manager.add_order(ExpressOrder("002", "Los Angeles", 5))
manager.add_order(BulkOrder("003", "Chicago", 100))
manager.process_orders()


Calculating optimized route for standard order 001 to New York
Order 001: delivery time is 2 days, capacity 10
Calculating generic route for order 002 to Los Angeles
Calculating delivery time for express order 002
Order 002: delivery time is 1 days, capacity 5
Calculating generic route for order 003 to Chicago
Calculating capacity for bulk order 003
Order 003: delivery time is 2 days, capacity 200


**Проблемы:**

1. **Нарушение SRP:**
   - Один класс отвечает за маршруты, время доставки и вместимость.

2. **LSP:**
   - Специальное поведение подклассов нарушает ожидания общего интерфейса.

**Решение:**

На самом деле, исходный код вполне себе неплохой и хорошо справляется с задачей уменьшения дублирования. Тем не менее, мы введем абстрактный класс и как минимум поменяем сообщение в calculate_route, что может быть полезным.

In [ ]:
from abc import ABC, abstractmethod


class Order(ABC):
    def __init__(self, order_id, destination, weight):
        self.order_id = order_id
        self.destination = destination
        self.weight = weight

    @abstractmethod
    def calculate_route(self):
        pass

    @abstractmethod
    def calculate_delivery_time(self):
        pass

    @abstractmethod
    def calculate_capacity(self):
        pass


class StandardOrder(Order):
    def calculate_route(self):
        print(f"Calculating optimized route for standard order {self.order_id} to {self.destination}")

    def calculate_delivery_time(self):
        return 2  # Время в днях

    def calculate_capacity(self):
        return self.weight


class ExpressOrder(Order):
    def calculate_route(self):
        print(f"Calculating generic route for express order {self.order_id} to {self.destination}")

    def calculate_delivery_time(self):
        print(f"Calculating delivery time for express order {self.order_id}")
        return 1  # Экспресс-доставка за 1 день

    def calculate_capacity(self):
        return self.weight


class BulkOrder(Order):
    def calculate_route(self):
        print(f"Calculating generic route for bulk order {self.order_id} to {self.destination}")

    def calculate_delivery_time(self):
        return 3  # Крупногабаритные заказы требуют больше времени

    def calculate_capacity(self):
        print(f"Calculating capacity for bulk order {self.order_id}")
        return self.weight * 2  # Специальный расчет для крупногабаритных заказов


class LogisticsManager:
    def __init__(self):
        self.orders = []

    def add_order(self, order: Order):
        self.orders.append(order)

    def process_orders(self):
        for order in self.orders:
            order.calculate_route()
            delivery_time = order.calculate_delivery_time()
            capacity = order.calculate_capacity()
            print(f"Order {order.order_id}: delivery time is {delivery_time} days, capacity {capacity}")


# Пример использования
manager = LogisticsManager()
manager.add_order(StandardOrder("001", "New York", 10))
manager.add_order(ExpressOrder("002", "Los Angeles", 5))
manager.add_order(BulkOrder("003", "Chicago", 100))
manager.process_orders()


Calculating optimized route for standard order 001 to New York
Order 001: delivery time is 2 days, capacity 10
Calculating generic route for express order 002 to Los Angeles
Calculating delivery time for express order 002
Order 002: delivery time is 1 days, capacity 5
Calculating generic route for bulk order 003 to Chicago
Calculating capacity for bulk order 003
Order 003: delivery time is 3 days, capacity 200


5. **Система обработки задач (5б)**

*Выполнять локально (не в colab).*

Разрабатываем систему обработки задач, использующую два типа очередей:
1. **Локальная очередь** — хранит задачи в оперативной памяти.
2. **Распределенная очередь** — взаимодействует с **RabbitMQ**.

Система выполняет три типа задач:
- `send_email`: Отправка email.
- `generate_report`: Генерация отчета.
- `process_data`: Обработка данных.


Для работы с **RabbitMQ** потребуется:
- Установленный и запущенный сервер RabbitMQ.
- Брокер сообщений будет доступен на порту `5672` для взаимодействия через AMQP-протокол.
- Веб-интерфейс управления RabbitMQ будет доступен по адресу [http://localhost:15672](http://localhost:15672), если вы используете официальный Docker-образ с поддержкой веб-интерфейса.

Чтобы запустить RabbitMQ, вы можете воспользоваться Docker. Это самый простой способ:

```bash
docker run -d --hostname my-rabbit --name some-rabbit \
-p 5672:5672 -p 15672:15672 rabbitmq:management
```

Если образ уже обновился, выполните команду для загрузки последней версии:

```bash
docker pull rabbitmq:management
```

После запуска:
- **Порт 5672** будет доступен для AMQP-протокола.
- **Порт 15672** будет доступен для веб-интерфейса RabbitMQ Management Console.
- Логин и пароль по умолчанию: `guest` / `guest`.

Убедитесь, что контейнер с RabbitMQ работает, выполнив команду.

```bash
docker ps
```

Перейдите в браузере на [http://localhost:15672](http://localhost:15672) для входа в веб-интерфейс с учетными данными `guest/guest`..

Исходный код:

In [ ]:
! pip install pika


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
class TaskProcessor:
    def process_task(self, task):
        task_type = task.get("type")

        payload = task.get("payload")

        if task_type == "send_email":
            self._send_email(payload)
        elif task_type == "generate_report":
            self._generate_report(payload)
        elif task_type == "process_data":
            self._process_data(payload)
        else:
            print(f"Unknown task type: {task_type}")

    def _send_email(self, payload):
        email = payload.get("email")
        subject = payload.get("subject")
        content = payload.get("content")
        print(f"Sending email to {email} with subject '{subject}' and content '{content}'")

    def _generate_report(self, payload):
        report_id = payload.get("report_id")
        print(f"Generating report with ID {report_id}...")
        # Имитация долгой работы
        import time
        time.sleep(2)
        print(f"Report {report_id} generated")

    def _process_data(self, payload):
        data = payload.get("data")
        print(f"Processing data: {data}")
        # Имитация обработки данных
        processed_data = [d.upper() for d in data]
        print(f"Processed data: {processed_data}")


In [ ]:
import time
import json
import pika


class TaskQueue:
    def enqueue(self, task):
        pass

    def dequeue(self):
        pass

    def connect(self):
        pass


class LocalQueue(TaskQueue):
    def __init__(self):
        self.tasks = []

    def enqueue(self, task):
        self.tasks.append(task)

    def dequeue(self):
        if not self.tasks:
            return None
        return self.tasks.pop(0)


class RabbitMQQueue(TaskQueue):
    def __init__(self, broker_url, queue_name):
        self.broker_url = broker_url
        self.queue_name = queue_name
        self.channel = None

    def connect(self):
        connection = pika.BlockingConnection(pika.ConnectionParameters(self.broker_url))
        self.channel = connection.channel()
        self.channel.queue_declare(queue=self.queue_name, durable=True)
        print(f"Connected to RabbitMQ queue '{self.queue_name}'")

    def enqueue(self, task):
        """Добавляет задачу в RabbitMQ с сериализацией в JSON."""
        if not self.channel:
            raise ConnectionError("Not connected to RabbitMQ")
        task_str = json.dumps(task)  # Сериализуем задачу в строку
        self.channel.basic_publish(
            exchange="",
            routing_key=self.queue_name,
            body=task_str,
            properties=pika.BasicProperties(
                delivery_mode=2  # Делает сообщение устойчивым
            )
        )
        print(f"Task '{task_str}' enqueued in RabbitMQQueue")

    def dequeue(self):
        """Извлекает задачу из RabbitMQ с десериализацией из JSON."""
        if not self.channel:
            raise ConnectionError("Not connected to RabbitMQ")
        method_frame, header_frame, body = self.channel.basic_get(queue=self.queue_name)
        if method_frame:
            self.channel.basic_ack(method_frame.delivery_tag)
            task = json.loads(body.decode())  # Десериализуем задачу обратно в словарь
            print(f"Task '{task}' dequeued from RabbitMQQueue")
            return task
        return None


class TaskManager:
    def __init__(self, queue, processor):
        self.queue = queue
        self.processor = processor

    def process_tasks(self):
        while True:
            task = self.queue.dequeue()
            if task is None:
                print("No tasks to process. Waiting...")
                import time
                time.sleep(2)
            else:
                print(f"Processing task: {task}")
                self.processor.process_task(task)


In [ ]:
local_queue = LocalQueue()
processor = TaskProcessor()
manager = TaskManager(local_queue, processor)

# Добавляем задачи в локальную очередь
local_queue.enqueue({"type": "send_email", "payload": {"email": "user@example.com", "subject": "Welcome", "content": "Thank you for joining!"}})
local_queue.enqueue({"type": "generate_report", "payload": {"report_id": 42}})
local_queue.enqueue({"type": "process_data", "payload": {"data": ["foo", "bar", "baz"]}})

# Запускаем обработку задач
manager.process_tasks()


Processing task: {'type': 'send_email', 'payload': {'email': 'user@example.com', 'subject': 'Welcome', 'content': 'Thank you for joining!'}}
Sending email to user@example.com with subject 'Welcome' and content 'Thank you for joining!'
Processing task: {'type': 'generate_report', 'payload': {'report_id': 42}}
Generating report with ID 42...
Report 42 generated
Processing task: {'type': 'process_data', 'payload': {'data': ['foo', 'bar', 'baz']}}
Processing data: ['foo', 'bar', 'baz']
Processed data: ['FOO', 'BAR', 'BAZ']
No tasks to process. Waiting...
No tasks to process. Waiting...
No tasks to process. Waiting...


KeyboardInterrupt: 

In [ ]:
# Создаем очередь RabbitMQ
rabbit_queue = RabbitMQQueue("localhost", "task_queue")
rabbit_queue.connect()

# Создаем обработчик задач и менеджер
processor = TaskProcessor()
manager = TaskManager(rabbit_queue, processor)

# Добавляем задачи в очередь
rabbit_queue.enqueue({"type": "send_email", "payload": {"email": "admin@example.com", "subject": "Alert", "content": "Server is down!"}})
rabbit_queue.enqueue({"type": "generate_report", "payload": {"report_id": 101}})
rabbit_queue.enqueue({"type": "process_data", "payload": {"data": ["apple", "banana", "cherry"]}})

# Запускаем обработку задач
manager.process_tasks()


Connected to RabbitMQ queue 'task_queue'
Task '{"type": "send_email", "payload": {"email": "admin@example.com", "subject": "Alert", "content": "Server is down!"}}' enqueued in RabbitMQQueue
Task '{"type": "generate_report", "payload": {"report_id": 101}}' enqueued in RabbitMQQueue
Task '{"type": "process_data", "payload": {"data": ["apple", "banana", "cherry"]}}' enqueued in RabbitMQQueue
Task '{'type': 'send_email', 'payload': {'email': 'admin@example.com', 'subject': 'Alert', 'content': 'Server is down!'}}' dequeued from RabbitMQQueue
Processing task: {'type': 'send_email', 'payload': {'email': 'admin@example.com', 'subject': 'Alert', 'content': 'Server is down!'}}
Sending email to admin@example.com with subject 'Alert' and content 'Server is down!'
Task '{'type': 'generate_report', 'payload': {'report_id': 101}}' dequeued from RabbitMQQueue
Processing task: {'type': 'generate_report', 'payload': {'report_id': 101}}
Generating report with ID 101...
Report 101 generated
Task '{'type':

KeyboardInterrupt: 

**Проблемы:**

1. **Нарушение SRP:**
   - Класс `TaskProcessor` выполняет разные задачи: обработка email, генерация отчётов и обработка данных.

2. **Нарушение OCP:**
   - Добавление новых типов задач потребует изменения в методе `process_task`.

3. **Нарушение DIP:**
   - Зависимость от конкретной реализации очередей (`LocalQueue`, `RabbitMQQueue`) в менеджере задач.

**Решение:**

In [ ]:
from abc import ABC, abstractmethod
import json
import pika
import time

class TaskHandler(ABC):
    @abstractmethod
    def handle(self, payload):
        pass

class EmailTaskHandler(TaskHandler):
    def handle(self, payload):
        email = payload.get("email")
        subject = payload.get("subject")
        content = payload.get("content")
        print(f"Sending email to {email} with subject '{subject}' and content '{content}'")

class ReportTaskHandler(TaskHandler):
    def handle(self, payload):
        report_id = payload.get("report_id")
        print(f"Generating report with ID {report_id}...")
        time.sleep(2)
        print(f"Report {report_id} generated")

class DataTaskHandler(TaskHandler):
    def handle(self, payload):
        data = payload.get("data")
        print(f"Processing data: {data}")
        processed_data = [d.upper() for d in data]
        print(f"Processed data: {processed_data}")

class TaskProcessor:
    def __init__(self):
        self.handlers = {
            "send_email": EmailTaskHandler(),
            "generate_report": ReportTaskHandler(),
            "process_data": DataTaskHandler()
        }

    def process_task(self, task):
        task_type = task.get("type")
        handler = self.handlers.get(task_type)
        if handler:
            handler.handle(task.get("payload"))
        else:
            print(f"Unknown task type: {task_type}")

class TaskQueue(ABC):
    @abstractmethod
    def enqueue(self, task):
        pass

    @abstractmethod
    def dequeue(self):
        pass

class LocalQueue(TaskQueue):
    def __init__(self):
        self.tasks = []

    def enqueue(self, task):
        self.tasks.append(task)

    def dequeue(self):
        if not self.tasks:
            return None
        return self.tasks.pop(0)

class RabbitMQQueue(TaskQueue):
    def __init__(self, broker_url, queue_name):
        self.broker_url = broker_url
        self.queue_name = queue_name
        self.channel = None

    def connect(self):
        connection = pika.BlockingConnection(pika.ConnectionParameters(self.broker_url))
        self.channel = connection.channel()
        self.channel.queue_declare(queue=self.queue_name, durable=True)
        print(f"Connected to RabbitMQ queue '{self.queue_name}'")

    def enqueue(self, task):
        if not self.channel:
            raise ConnectionError("Not connected to RabbitMQ")
        task_str = json.dumps(task)
        self.channel.basic_publish(
            exchange="",
            routing_key=self.queue_name,
            body=task_str,
            properties=pika.BasicProperties(delivery_mode=2)
        )
        print(f"Task '{task_str}' enqueued in RabbitMQQueue")

    def dequeue(self):
        if not self.channel:
            raise ConnectionError("Not connected to RabbitMQ")
        method_frame, header_frame, body = self.channel.basic_get(queue=self.queue_name)
        if method_frame:
            self.channel.basic_ack(method_frame.delivery_tag)
            task = json.loads(body.decode())
            print(f"Task '{task}' dequeued from RabbitMQQueue")
            return task
        return None

class TaskManager:
    def __init__(self, queue, processor):
        self.queue = queue
        self.processor = processor

    def process_tasks(self):
        while True:
            task = self.queue.dequeue()
            if task is None:
                print("No tasks to process. Waiting...")
                time.sleep(2)
            else:
                print(f"Processing task: {task}")
                self.processor.process_task(task)


local_queue = LocalQueue()
processor = TaskProcessor()
manager = TaskManager(local_queue, processor)

local_queue.enqueue({"type": "send_email", "payload": {"email": "user@example.com", "subject": "Welcome", "content": "Thank you for joining!"}})
local_queue.enqueue({"type": "generate_report", "payload": {"report_id": 42}})
local_queue.enqueue({"type": "process_data", "payload": {"data": ["foo", "bar", "baz"]}})

manager.process_tasks()

rabbit_queue = RabbitMQQueue("localhost", "task_queue")
rabbit_queue.connect()

rabbit_queue.enqueue({"type": "send_email", "payload": {"email": "admin@example.com", "subject": "Alert", "content": "Server is down!"}})
rabbit_queue.enqueue({"type": "generate_report", "payload": {"report_id": 101}})
rabbit_queue.enqueue({"type": "process_data", "payload": {"data": ["apple", "banana", "cherry"]}})

manager = TaskManager(rabbit_queue, processor)
manager.process_tasks()

Проверьте в веб-интерфейсе, что у вас появилась очередь сообщений `task_queue` и в ней было какое-то сообщение.

После нескольких секунд для каждого обработчика можно его прерывать, поскольку каждый из них будет просто ожидать данных и писать `No tasks to process. Waiting...`

Для удаления Docker-образа после использования вам нужно выполнить несколько шагов:

#### 1. Остановить и удалить контейнер

Если контейнер с RabbitMQ всё ещё работает, его нужно сначала остановить и удалить:

```bash
docker ps
```

Эта команда покажет список всех работающих контейнеров. Найдите контейнер RabbitMQ, например:

```
CONTAINER ID   IMAGE                COMMAND                  STATUS        NAMES
e0db3c8c0d8f   rabbitmq:management  "docker-entrypoint.s…"   Up 3 minutes  some-rabbit
```

Чтобы остановить контейнер, выполните:

```bash
docker stop some-rabbit
```

Теперь удалите остановленный контейнер:

```bash
docker rm some-rabbit
```

#### 2. Удалить Docker-образ

Сначала проверьте, какие образы загружены в вашей системе:

```bash
docker images
```

Вы увидите список образов:

```
REPOSITORY     TAG            IMAGE ID       CREATED        SIZE
rabbitmq       management     f41c9db5c2c5   2 weeks ago    188MB
```

Удалите образ RabbitMQ:

```bash
docker rmi rabbitmq:management
```


P.S. Для общего развития рекомендуется почитать про брокеры сообщений